In [88]:
import pandas as pd
import numpy as np
import openpyxl
import fastparquet

In [89]:
sheets = [
    {
        "name": "Table 2",
        "type": "Division",
        "skiprows": 6,
        "cols": ["description", "code"],
        "measure": "Value £ million"
    },
    {
        "name": "Table 3",
        "type": "Industry",
        "skiprows": 6,
        "cols": ["parent_description", "code", "description"],
        "measure": "Value £ million"
    },
    {
        "name": "Table 5",
        "type": "Product",
        "skiprows": 5,
        "cols": ["code", "description", "measure"]
    },
]

dfs = []
for sheet in sheets:
    df = pd.read_excel(
        "prodcom_accessiblepublicationtables2024.xlsx",
        sheet_name=sheet["name"],
        skiprows=sheet["skiprows"],
        dtype=str
    )
    df["type"] = sheet["type"]
    df.columns = sheet["cols"] + list(df.columns[len(sheet["cols"]):])
    if "measure" in sheet:
        df["measure"] = sheet["measure"]
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

# Keep the description, code, parent_description and years cols
df = df[["description", "code", "parent_description", "type", "measure"] + [col for col in df.columns if col.startswith("20")]]
# Rename the measure to unit
df = df.rename(columns={"measure": "unit"})

df = df.melt(id_vars=["description", "code", "parent_description", "type", "unit"], var_name="year", value_name="value")

# Extract any [square bracketed text] from the value to get the flags
df['flag'] = df['value'].str.extract(r'\[(.*?)\]')
df.flag.value_counts(dropna=False)


# # Remove the flags and commas from the value and convert to numeric
df['value'] = df['value'].astype(str).str.replace(r'\[.*?\]', '', regex=True)
df['value'] = df['value'].astype(str).str.replace(r',', '', regex=True)
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# A value of zero in the data corresponds to no data collected, add this to the flag
df['flag'] = np.where(df['value']==0,"0",df['flag'])

# To keep flags for missing data, set all nan values to 0 instead of dropping OR keep nan values altogether
df = df.dropna(subset=['value'])
# df['value']=df['value'].fillna(0)

df['flag'] = df['flag'].map({
    "e": "e - low response; high level of estimation",
    "c": "c - data suppressed; data is disclosive",
    "a": "a - data suppressed; data is disclosive and is aggregated within the UK Manufacturer Sales of 'Other' products",
    "x": "x - data not available",
    "0": "No data collected"
})

# Use 'measure' to group by value, volume, or average price
df['measure'] = np.where(df['unit'].str.contains('Value', case=False), 'Value',
                         np.where(df['unit'].str.contains('Volume', case=False), 'Volume',
                                  'Average price/Other'))

df['code'] = df.code.astype(str)

df.to_parquet("prodcom.parquet", index=False)

In [4]:
df[["description", "code", "type"]].drop_duplicates().to_json("cn8_division_industry_product.json", orient="records", force_ascii=False)

In [61]:
df.columns

Index(['description', 'code', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021', '2022', '2023', '2024', 'type', 'measure',
       'parent_description', 'notes'],
      dtype='object')

In [10]:
df.measure.unique()

array(['Value', 'Volume', 'Average price/Other'], dtype=object)

In [9]:
df.year.unique()

array(['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023', '2024'], dtype=object)

In [7]:
df.type.unique()

array(['Division', 'Industry', 'Product'], dtype=object)

In [52]:
df.query("code == -1")

AttributeError: 'UnaryOp' object has no attribute 'evaluate'

In [48]:
# What non-numeric codes do we have?
df[df.code.astype(str).str.contains(r'\D')].code.unique()

array(['All Divisions', 'Industry Total', '11 & 12'], dtype=object)

In [42]:
df.sample(100)

,description,code,parent_description,type,unit,year,value,flag,measure
23275,10122080 (CN 02071491 + 02072791 + 02074593 +...,10122080,NaN,Product,£ per Kilogram,2016,8.500000e-01,NaN,Average price/Other
68334,"Manufacture of electric motors, generators and...",27110,Manufacture of electrical equipment,Industry,Value £ million,2020,1.294000e+03,NaN,Value
28256,"22231290 (CN 392290), Plastic bidets; lavator...",22231290,NaN,Product,£ per Number of items,2016,6.470000e+00,NaN,Average price/Other
56967,Manufacture of electronic components,26110,Manufacture of computer; electronic and optica...,Industry,Value £ million,2019,1.111000e+03,NaN,Value
66802,"28921100 (CN 842831), Continuous-action eleva...",28921100,NaN,Product,Value £000's,2019,5.209400e+04,NaN,Value
...,...,...,...,...,...,...,...,...,...
98384,"26112180 (CN 854130), Semiconductor thyristor...",26112180,NaN,Product,£ per Number of items,2022,1.059700e+02,NaN,Average price/Other
16978,"22299160, Plastic parts and accessories for al...",22299160,NaN,Product,Value £000's,2015,1.225444e+06,NaN,Value
114335,"10311460 (CN 20052020 + 20052080), Potatoes p...",10311460,NaN,Product,Volume (Kilogram),2024,1.888676e+08,NaN,Volume
9907,"28491150 (CN 846420), Grinding or polishing m...",28491150,NaN,Product,£ per Number of items,2014,1.377465e+04,NaN,Average price/Other


In [16]:
pd.DataFrame(df.measure.value_counts(dropna=False)).head(30)

,count
measure,
Value £000's,4051
Volume (Kilogram),1587
£ per Kilogram,1587
Volume (Number of items),1197
£ per Number of items,1197
Volume (Tonnes),368
£ per Tonnes,368
Value £ million,258
£ per Square metre,97


In [ ]:
df.measu

In [7]:
dfs[2]

,08111133,"08111133 (CN 251511), Marble and travertine, crude or roughly trimmed",Value £000's,[a],0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,Unnamed: 14,type
0,8111133,"08111133 (CN 251511), Marble and travertine, ...",Volume (Tonnes),[c],0,0,0,0,0,0,0,0,0,0,NaN,Product
1,8111133,"08111133 (CN 251511), Marble and travertine, ...",£ per Tonnes,[c],[x],[x],[x],[x],[x],[x],[x],[x],[x],[x],NaN,Product
2,8111136,"08111136 (CN 251512), Marble and travertine m...",Value £000's,[a],0,0,0,0,0,0,0,0,0,0,NaN,Product
3,8111136,"08111136 (CN 251512), Marble and travertine m...",Volume (Tonnes),[c],0,0,0,0,0,0,0,0,0,0,NaN,Product
4,8111136,"08111136 (CN 251512), Marble and travertine m...",£ per Tonnes,[c],[x],[x],[x],[x],[x],[x],[x],[x],[x],[x],NaN,Product
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11093,33205050,"33205050, Installation of electricity distrib...",Value £000's,243789,265229,237617,213739,246739,220894,219153,107084,137971,87059,90775,NaN,Product
11094,33205090,"33205090, Installation of electrical equipmen...",Value £000's,18750,19263,20476,14595,14756,15485,72776,75425,52829,85306,88289,NaN,Product
11095,33206000,"33206000, Installation of industrial process ...",Value £000's,934563,849463,750552,655150,438710,456097,408604,645473,538092,562079,447829,NaN,Product
11096,33207000,"33207000, Installation of non-domestic time m...",Value £000's,778,686,588,934,1288,1449,1639,2354,1439,[a],[a],NaN,Product
